# © Artur Czarnecki. All rights reserved.
# Integrax framework

# Notebook 11 — ChatGPT-like E2E (behavior-level)

This notebook is an **integration / behavior** test that exercises the Drop-In Knowledge Runtime end-to-end, in a way that resembles a real ChatGPT usage pattern.

## What we test (behavior)
- Multi-session behavior (A/B/C) with **isolated session history** and **shared user LTM**
- User LTM persistence + recall across sessions
- Session-level consolidation (history → summary) without cross-session leakage
- RAG ingestion + Q&A over a document
- Websearch as a context layer that affects the final answer
- Tools execution (tool + LLM) without breaking the **user-last invariant**
- Reasoning enabled for observability, but **not persisted into user-visible history**

## What we do NOT test (intentionally out of scope for this notebook)
- Retry / fallback logic for empty LLM outputs
- Formal “adapter contract” beyond `generate_messages(...) -> str`
- Tools contract v1 (final answer vs context-only)
- Prompt tuning, quality scoring, reranking
- Debug refactors (e.g., removing getattr) unless strictly required to run the notebook

## Hard invariants
- **User-last invariant:** the last message sent to the core LLM must always be `role="user"`
- No empty assistant answers in the final output
- No memory leakage between sessions (history isolation)

## Requirements
This notebook expects your local environment to provide:
- Intergrax sources on PYTHONPATH
- LLM credentials (e.g., `OPENAI_API_KEY`) if using OpenAI adapters
- Vectorstore backend deps (Chroma/Qdrant) if enabling RAG/LTM vector retrieval


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_CSE_API_KEY"] = os.getenv("GOOGLE_CSE_API_KEY") or ""

In [3]:
import os, sys
from datetime import datetime
from pathlib import Path
import uuid
from intergrax.llm_adapters.base import LLMAdapter, LLMAdapterRegistry, LLMProvider
from intergrax.runtime.drop_in_knowledge_mode.config import ReasoningConfig, RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager
from intergrax.memory.user_profile_manager import UserProfileManager
from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.vectorstore_manager import VSConfig, VectorstoreManager
from intergrax.runtime.user_profile.session_memory_consolidation_service import SessionMemoryConsolidationService
from intergrax.runtime.user_profile.user_profile_instructions_service import UserProfileInstructionsService
from intergrax.websearch.providers.google_cse_provider import GoogleCSEProvider
from intergrax.websearch.service.websearch_executor import WebSearchExecutor

# =====================================================================
# Global test identifiers / paths (no tests executed in this cell)
# =====================================================================

USER_ID = "user_chatgpt_like_001"

SESSION_A = "sess_chatgpt_like_A"
SESSION_B = "sess_chatgpt_like_B"
SESSION_C = "sess_chatgpt_like_C"

RUN_ID = datetime.utcnow().strftime("%Y%m%d_%H%M%S") + "_" + uuid.uuid4().hex[:8]

BASE_DIR = Path(os.getcwd()).resolve()
ARTIFACTS_DIR = BASE_DIR / "_artifacts" / "notebook_11_chatgpt_like" / RUN_ID
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

# Separate vectorstore collections (1 instance = 1 collection_name)
# - RAG_DOCS: document ingestion/retrieval
# - USER_LTM: user long-term memory retrieval
RAG_DIR = ARTIFACTS_DIR / "vs_rag_docs"
LTM_DIR = ARTIFACTS_DIR / "vs_user_ltm"
RAG_DIR.mkdir(parents=True, exist_ok=True)
LTM_DIR.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------------------
# LLM adapter (real adapter, no wrappers)
# - assumes env is configured (OPENAI_API_KEY etc.)
# ---------------------------------------------------------------------
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)


# ---------------------------------------------------------------------
# Embeddings + vectorstore (real managers)
# Pick providers you actually use in your repo/env.
# ---------------------------------------------------------------------

embed_manager = EmbeddingManager(
    provider="ollama",
)

rag_vs = VectorstoreManager(
    config=VSConfig(
        provider="chroma",
        collection_name=f"rag_docs_{RUN_ID}",
        chroma_persist_directory=str(RAG_DIR),
    )
)

# User LTM vectorstore
ltm_vs = VectorstoreManager(
    VSConfig(
        provider="chroma",
        collection_name=f"user_ltm_{RUN_ID}",
        chroma_persist_directory=str(LTM_DIR),
    )
)

# ---------------------------------------------------------------------
# Stores
# ---------------------------------------------------------------------
session_store = InMemorySessionStorage()
user_profile_store = InMemoryUserProfileStore()

# ---------------------------------------------------------------------
# Managers
# ---------------------------------------------------------------------
user_profile_manager = UserProfileManager(
    store=user_profile_store,
    embedding_manager=embed_manager,
    vectorstore_manager=ltm_vs,
)

user_profile_instructions_service = UserProfileInstructionsService(
    llm=llm_adapter,
    manager=user_profile_manager,
)

session_memory_consolidation_service = SessionMemoryConsolidationService(
    llm=llm_adapter,
    profile_manager=user_profile_manager,
    instructions_service=user_profile_instructions_service,
)


session_manager = SessionManager(
    storage=session_store,
    user_profile_manager=user_profile_manager,
    session_memory_consolidation_service=session_memory_consolidation_service
)


websearch_executor = WebSearchExecutor(
    providers=[
        GoogleCSEProvider(),
    ],
    max_text_chars=None,
)

# ---------------------------------------------------------------------
# Runtime config
# ---------------------------------------------------------------------
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    embedding_manager=embed_manager,
    vectorstore_manager=rag_vs,
    websearch_executor=websearch_executor,
    enable_user_profile_memory=True,
    enable_org_profile_memory=False,
    enable_user_longterm_memory=True,
    enable_rag=True,
    enable_websearch=True,
    tools_mode="off",
    reasoning_config=ReasoningConfig()
)


# =====================================================================
# Runtime factory (keeps Cell-2..Cell-8 focused on behavior)
# =====================================================================
def build_runtime(*, override_config: dict | None = None, **runtime_kwargs) -> DropInKnowledgeRuntime:
    """
    Build a DropInKnowledgeRuntime instance.
    - override_config: dict of RuntimeConfig fields to override (shallow).
    - runtime_kwargs: runtime init kwargs (e.g., ingestion_service, context_builder, prompt builders).
    """
    if override_config:
        for k, v in override_config.items():
            setattr(config, k, v)

    return DropInKnowledgeRuntime(
        config=config,
        session_manager=session_manager,
        ingestion_service=runtime_kwargs.get("ingestion_service"),
        context_builder=runtime_kwargs.get("context_builder"),
        rag_prompt_builder=runtime_kwargs.get("rag_prompt_builder"),
        websearch_prompt_builder=runtime_kwargs.get("websearch_prompt_builder"),
        history_prompt_builder=runtime_kwargs.get("history_prompt_builder"),
    )


print("BOOTSTRAP OK")
print("RUN_ID:", RUN_ID)
print("ARTIFACTS_DIR:", str(ARTIFACTS_DIR))
print("USER_ID:", USER_ID)
print("SESSION_A:", SESSION_A)
print("SESSION_B:", SESSION_B)
print("SESSION_C:", SESSION_C)


C:\Users\XPS\AppData\Local\Temp\ipykernel_40832\2251295334.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  RUN_ID = datetime.utcnow().strftime("%Y%m%d_%H%M%S") + "_" + uuid.uuid4().hex[:8]


[intergraxVectorstoreManager] Initialized provider=chroma, collection=rag_docs_20251223_091134_741683ee
[intergraxVectorstoreManager] Existing count: 0
[intergraxVectorstoreManager] Initialized provider=chroma, collection=user_ltm_20251223_091134_741683ee
[intergraxVectorstoreManager] Existing count: 0
BOOTSTRAP OK
RUN_ID: 20251223_091134_741683ee
ARTIFACTS_DIR: D:\Projekty\intergrax\notebooks\drop_in_knowledge_mode\_artifacts\notebook_11_chatgpt_like\20251223_091134_741683ee
USER_ID: user_chatgpt_like_001
SESSION_A: sess_chatgpt_like_A
SESSION_B: sess_chatgpt_like_B
SESSION_C: sess_chatgpt_like_C


## Cell 2 — Session A: onboarding + LTM write

Goal:
- Simulate a real onboarding turn (user introduces themselves and preferences).
- Run the full runtime pipeline via `runtime.run(...)`.
- Close the session to trigger **session → LTM consolidation**.
- Minimal asserts:
  1) Assistant answer is non-empty
  2) Session history is persisted
  3) User LTM contains at least one entry after closing the session


In [4]:
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest

# ---------------------------------------------------------------------
# Build runtime (no special overrides yet)
# ---------------------------------------------------------------------
runtime = build_runtime()

# ---------------------------------------------------------------------
# Session A — onboarding message (behaves like real ChatGPT)
# ---------------------------------------------------------------------
onboarding_message = (
    "Hi. I am Artur. I build Integrax and Mooff. "
    "I prefer concise, technical answers. Never use emojis in code or technical docs. "
    "Please remember this for future sessions."
)

request_a = RuntimeRequest(
    user_id=USER_ID,
    session_id=SESSION_A,
    message=onboarding_message,
)

answer_a = await runtime.run(request_a)

# ---------------------------------------------------------------------
# Minimal assert #1: non-empty assistant answer
# ---------------------------------------------------------------------
assert isinstance(answer_a.answer, str) and answer_a.answer.strip(), "Empty assistant answer in Session A."

# ---------------------------------------------------------------------
# Minimal assert #2: session history persisted (via SessionManager storage)
# ---------------------------------------------------------------------
history_a = await session_manager.get_history(session_id=SESSION_A)
assert len(history_a) >= 2, f"Expected >=2 messages in history, got {len(history_a)}."

# ---------------------------------------------------------------------
# Trigger consolidation to LTM by closing the session
# (This is the behavior boundary: Session -> LTM)
# ---------------------------------------------------------------------
await session_manager.close_session(session_id=SESSION_A)

# ---------------------------------------------------------------------
# Minimal assert #3: LTM entries were created (semantic recall evidence)
# We do a vector search against the user's LTM store.
# ---------------------------------------------------------------------
ltm_search = await user_profile_manager.search_longterm_memory(
        user_id=USER_ID,
        query="Artur Integrax Mooff preferences concise technical no emojis",
        top_k=5,
        score_threshold=0.0,
    )

assert ltm_search.get("used_longterm") is True, "Expected long-term memory retrieval to be enabled."
assert (ltm_search.get("debug") or {}).get("hits_count", 0) > 0, "Expected at least one LTM entry after closing Session A."

print("SESSION A OK")
print("Answer length:", len(answer_a.answer))
print("History messages:", len(history_a))
print("LTM hits_count:", ltm_search.get("hits_count"))
print("LTM debug:", ltm_search.get("debug"))
print("LTM hits:", len(ltm_search.get("hits") or []))


[intergraxVectorstoreManager] Upserting 1 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 1
[intergraxVectorstoreManager] Upserting 1 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 2
[intergraxVectorstoreManager] Upserting 1 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 3
SESSION A OK
Answer length: 331
History messages: 2
LTM hits_count: None
LTM debug: {'enabled': True, 'used': True, 'reason': 'hits', 'where': {'user_id': 'user_chatgpt_like_001', 'deleted': 0}, 'top_k': 5, 'threshold': 0.0, 'raw_ids': ['5e8d35195aa8467db8959b416f6477bf', '3619e58acfc64a458f97dbd45b99682b', '91205505565f42508ce18a0760c3084d'], 'raw_scores': [0.23801147937774658, 0.1308668851852417, 0.008136987686157227], 'raw_metadatas': [{'entry_id': '5e8d35195aa8467db8959b416f6477bf', 'tags': 'user,goal', 'user_id': 'user_chatgpt_like_001', 'source': 'session_consolida

## Cell 3 — Session B: recall (ChatGPT behavior)

Goal:
- Start a new session_id (fresh history).
- Ask the system to recall facts from Session A using User LTM.
- Minimal asserts:
  1) Non-empty assistant answer
  2) debug_trace shows User LTM was used
  3) Answer contains recalled facts/preferences
  4) Session B history is isolated from Session A


In [5]:
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest

runtime = build_runtime()

recall_prompt = (
    "Before we continue: remind me who I am and what I build. "
    "Also remind me what answer style I prefer."
)

request_b = RuntimeRequest(
    user_id=USER_ID,
    session_id=SESSION_B,
    message=recall_prompt,
)

answer_b = await runtime.run(request_b)

# 1) Non-empty answer
assert isinstance(answer_b.answer, str) and answer_b.answer.strip(), "Empty assistant answer in Session B."

# 2) Debug evidence: User LTM used
dbg = answer_b.debug_trace or {}
ltm_dbg = dbg.get("user_longterm_memory") or {}
ltm_used = bool(ltm_dbg.get("used") is True or ltm_dbg.get("used_longterm") is True)
assert ltm_used, f"Expected LTM to be used in Session B. user_longterm_memory={ltm_dbg}"

# 3) Behavior evidence: answer contains recalled facts/preferences
ans_norm = answer_b.answer.lower()
expected_any = [
    "artur",
    "intergrax",
    "mooff",
    "concise",
    "technical",
    "never use emojis",
    "no emojis",
]
assert any(k in ans_norm for k in expected_any), (
    "Expected the answer to include recalled facts/preferences from Session A. "
    f"Answer was:\n{answer_b.answer}"
)

# 4) Session history isolation sanity check
history_b = await session_manager.get_history(session_id=SESSION_B)
assert len(history_b) >= 2, f"Expected >=2 messages in Session B history, got {len(history_b)}."

# First user message in Session B should be the recall prompt (not Session A onboarding)
first_user_b = next((m for m in history_b if getattr(m, "role", None) == "user"), None)
assert first_user_b is not None, "Expected a user message in Session B history."
assert recall_prompt.strip() in (first_user_b.content or ""), "Session B history isolation issue."

print("SESSION B OK")
print("Answer length:", len(answer_b.answer))
print("LTM debug:", ltm_dbg)
print("History messages:", len(history_b))


SESSION B OK
Answer length: 598
LTM debug: {'enabled': True, 'used': True, 'reason': 'hits', 'where': {'user_id': 'user_chatgpt_like_001', 'deleted': 0}, 'top_k': 8, 'threshold': None, 'raw_ids': ['3619e58acfc64a458f97dbd45b99682b', '91205505565f42508ce18a0760c3084d', '5e8d35195aa8467db8959b416f6477bf'], 'raw_scores': [-0.26603829860687256, -0.37259626388549805, -0.49666833877563477], 'raw_metadatas': [{'entry_id': '3619e58acfc64a458f97dbd45b99682b', 'tags': 'session_summary', 'deleted': 0, 'source': 'session_consolidation', 'kind': 'session_summary', 'user_id': 'user_chatgpt_like_001'}, {'kind': 'preference', 'entry_id': '91205505565f42508ce18a0760c3084d', 'deleted': 0, 'tags': 'communication,tone', 'user_id': 'user_chatgpt_like_001', 'source': 'session_consolidation'}, {'deleted': 0, 'source': 'session_consolidation', 'user_id': 'user_chatgpt_like_001', 'entry_id': '5e8d35195aa8467db8959b416f6477bf', 'tags': 'user,goal', 'kind': 'user_fact'}], 'raw_documents_preview': ['Artur przedst

## Cell 4 — Ingestion + RAG (document Q&A)

Goal:
- Ingest a real document into the **RAG vectorstore** via the runtime attachment ingestion flow.
- Ask a question that can be answered only from the document.
- Ask a second question that requires **RAG + User LTM** at the same time.

Minimal asserts:
1) Ingestion step completed (debug evidence)
2) RAG was used (debug evidence: rag.used or rag_chunks > 0)
3) Answer is non-empty and includes document facts
4) For the combined question: both RAG and LTM were used


In [6]:
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest
from intergrax.llm.messages import AttachmentRef
from intergrax.runtime.drop_in_knowledge_mode.ingestion.attachments import FileSystemAttachmentResolver
from intergrax.runtime.drop_in_knowledge_mode.ingestion.ingestion_service import AttachmentIngestionService

# ---------------------------------------------------------------------
# Create a real document file to ingest
# ---------------------------------------------------------------------
DOC_SESSION = "sess_chatgpt_like_RAG"

doc_path = ARTIFACTS_DIR / "rag_doc_001.md"
doc_text = """# Integrax — RAG Demo Document

This document is used for an E2E test of Drop-In Knowledge Runtime.

Key modules:
- Drop-In Knowledge Runtime
- User Long-Term Memory (LTM)
- Websearch

Important constants:
- The default max entries per LTM query is 8.
- The project codename for the demo is "NEBULA-11".

Behavior requirement:
- Answers must be concise and technical.
"""
doc_path.write_text(doc_text, encoding="utf-8")

attachment = AttachmentRef(
    id="rag_doc_001",
    type="md",
    uri=doc_path.as_uri(),          # raw path is supported by FileSystemAttachmentResolver
    metadata={"label": "RAG Demo Document"}
)

# ---------------------------------------------------------------------
# Build ingestion service (indexes into rag_vs)
# ---------------------------------------------------------------------
resolver = FileSystemAttachmentResolver()

ingestion_service = AttachmentIngestionService(
    resolver=resolver,
    embedding_manager=embed_manager,
    vectorstore_manager=rag_vs,     # IMPORTANT: documents go to RAG vectorstore
)

runtime = build_runtime(ingestion_service=ingestion_service)

# ---------------------------------------------------------------------
# Turn 1: Upload + ingestion
# ---------------------------------------------------------------------
request_ingest = RuntimeRequest(
    user_id=USER_ID,
    session_id=DOC_SESSION,
    message="I uploaded a document. Please ingest it and confirm.",
    attachments=[attachment],
)

answer_ingest = await runtime.run(request_ingest)

assert isinstance(answer_ingest.answer, str) and answer_ingest.answer.strip(), "Empty assistant answer after ingestion."

dbg_ingest = answer_ingest.debug_trace or {}
ing_dbg = dbg_ingest.get("ingestion") or {}

# Ingestion debug shape can vary, so keep it minimal:
# - either presence of ingestion trace
# - or a non-empty list of ingested results
assert ing_dbg is not None, f"Expected ingestion debug trace. debug_trace={dbg_ingest}"

print("INGEST OK")
print("Ingestion debug:", ing_dbg)

# ---------------------------------------------------------------------
# Turn 2: Pure document Q&A (RAG must be used)
# ---------------------------------------------------------------------
q_doc = "In the uploaded document: what is the demo codename and what are the three key modules?"

request_doc_qa = RuntimeRequest(
    user_id=USER_ID,
    session_id=DOC_SESSION,
    message=q_doc,
)

answer_doc = await runtime.run(request_doc_qa)

assert isinstance(answer_doc.answer, str) and answer_doc.answer.strip(), "Empty assistant answer in document Q&A."

dbg_doc = answer_doc.debug_trace or {}
rag_dbg = dbg_doc.get("rag") or {}
rag_chunks = dbg_doc.get("rag_chunks", 0)

assert (rag_dbg.get("used") is True) or (rag_chunks and rag_chunks > 0), (
    f"Expected RAG to be used. rag={rag_dbg}, rag_chunks={rag_chunks}"
)

ans_doc_norm = answer_doc.answer.lower()
assert ("nebula-11".lower() in ans_doc_norm), "Expected the answer to include the codename from the document."
assert ("drop-in" in ans_doc_norm) or ("long-term" in ans_doc_norm) or ("websearch" in ans_doc_norm), (
    "Expected the answer to include at least one key module from the document."
)

print("DOC QA OK")
print("Answer length:", len(answer_doc.answer))
print("RAG debug:", rag_dbg)
print("RAG chunks:", rag_chunks)

# ---------------------------------------------------------------------
# Turn 3: Combined question (RAG + LTM)
# - Must use RAG (document facts) and LTM (user preference / identity).
# ---------------------------------------------------------------------
q_combined = (
    "Using the uploaded document AND what you remember about me: "
    "write a concise technical answer that (1) states who I am and what I build, "
    "(2) lists the document's key modules, and (3) includes the codename."
)

request_combined = RuntimeRequest(
    user_id=USER_ID,
    session_id=DOC_SESSION,
    message=q_combined,
)

answer_combined = await runtime.run(request_combined)

assert isinstance(answer_combined.answer, str) and answer_combined.answer.strip(), "Empty assistant answer in combined RAG+LTM question."

dbg_combined = answer_combined.debug_trace or {}

# RAG evidence
rag_dbg2 = dbg_combined.get("rag") or {}
rag_chunks2 = dbg_combined.get("rag_chunks", 0)
rag_used2 = (rag_dbg2.get("used") is True) or (rag_chunks2 and rag_chunks2 > 0)
assert rag_used2, f"Expected RAG to be used in combined question. rag={rag_dbg2}, rag_chunks={rag_chunks2}"

# LTM evidence
ltm_dbg2 = dbg_combined.get("user_longterm_memory") or {}
ltm_used2 = bool(ltm_dbg2.get("used") is True or ltm_dbg2.get("used_longterm") is True)
assert ltm_used2, f"Expected LTM to be used in combined question. ltm={ltm_dbg2}"

# Behavior evidence: contains memory facts + doc codename
ans2 = answer_combined.answer.lower()
assert "artur" in ans2, "Expected the combined answer to include 'Artur' from LTM."
assert ("intergrax" in ans2) or ("mooff" in ans2), "Expected the combined answer to include Integrax/Mooff from LTM."
assert "nebula-11" in ans2, "Expected the combined answer to include the document codename."

print("COMBINED RAG+LTM OK")
print("Answer length:", len(answer_combined.answer))
print("RAG chunks:", rag_chunks2)
print("LTM debug:", ltm_dbg2)


[intergraxVectorstoreManager] Upserting 1 items (dim=1536) to provider=chroma...
[intergraxVectorstoreManager] Upsert complete. New count: 1
INGEST OK
Ingestion debug: [{'attachment_id': 'rag_doc_001', 'attachment_type': 'md', 'num_chunks': 1, 'vector_ids_count': 1, 'metadata': {'source_path': 'D:\\Projekty\\intergrax\\notebooks\\drop_in_knowledge_mode\\_artifacts\\notebook_11_chatgpt_like\\20251223_091134_741683ee\\rag_doc_001.md', 'session_id': 'sess_chatgpt_like_RAG', 'user_id': 'user_chatgpt_like_001', 'tenant_id': None, 'workspace_id': None}}]
DOC QA OK
Answer length: 176
RAG debug: {'enabled': True, 'used': True, 'hits_count': 1, 'where_filter': {'session_id': 'sess_chatgpt_like_RAG', 'user_id': 'user_chatgpt_like_001'}, 'top_k': 8, 'score_threshold': None, 'hits': [{'id': 'rag_doc_001-0', 'score': 0.9068, 'metadata': {'source_name': 'rag_doc_001.md', 'chunk_total': 1, 'attachment_type': 'md', 'attachment_id': 'rag_doc_001', 'source_path': 'D:\\Projekty\\intergrax\\notebooks\\dro

## Cell 5 — Websearch (current knowledge, no documents)

Goal:
- Ask a question that requires up-to-date knowledge.
- Ensure there are no documents in context (disable RAG for this cell).
- Verify that:
  1) Websearch was used (debug evidence)
  2) Websearch produced context blocks / sources
  3) The final answer references the retrieved web context (behavior evidence)

Minimal asserts:
- Non-empty assistant answer
- debug_trace.websearch.used == True (or equivalent)
- debug_trace.websearch.context_blocks_count > 0 (or sources_count > 0)


In [ ]:
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest
from intergrax.websearch.service.websearch_config import WebSearchConfig, WebSearchStrategyType

WEB_SESSION = "sess_chatgpt_like_WEB"

# Build runtime with RAG disabled but websearch enabled.
runtime_web = build_runtime(
    override_config={
        "enable_rag": False,
        "enable_websearch": True,
    }
)

# Ensure websearch strategy + adapters are configured for this runtime instance.
cfg = runtime_web._config  # assuming your build_runtime attaches config here (as in other notebook cells)
cfg.websearch_config = WebSearchConfig(strategy=WebSearchStrategyType.MAP_REDUCE)
cfg.websearch_config.llm.map_adapter = cfg.llm_adapter
cfg.websearch_config.llm.reduce_adapter = cfg.llm_adapter

web_q = (
    "What are the most recent major changes to the OpenAI API regarding the Responses API "
    "and tool calling? Provide a concise technical summary with the date of the change."
)

request_web = RuntimeRequest(
    user_id=USER_ID,
    session_id=WEB_SESSION,
    message=web_q,
)

answer_web = await runtime_web.run(request_web)

# 1) Non-empty answer (runtime answered)
assert isinstance(answer_web.answer, str) and answer_web.answer.strip(), "Empty assistant answer in Websearch session."

# 2) Routing says websearch was used
route = answer_web.route
assert route is not None, "Missing route in RuntimeAnswer."
assert route.used_websearch is True, f"Expected used_websearch=True. route={route}"

dbg = answer_web.debug_trace or {}
ws_dbg = dbg.get("websearch") or {}

# 3) Websearch produced context blocks + preview
ctx_blocks = int(ws_dbg.get("context_blocks_count", 0) or 0)
assert ctx_blocks > 0, f"Expected websearch context blocks > 0. websearch={ws_dbg}"

preview = (ws_dbg.get("context_preview") or "").strip()
assert preview, f"Expected non-empty websearch context preview. websearch={ws_dbg}"

# 4) Diagnostics: raw results preview should exist (ensures executor returned objects with url/title)
raw_preview = ws_dbg.get("raw_results_preview") or []
assert isinstance(raw_preview, list), f"Expected raw_results_preview list. websearch={ws_dbg}"
assert len(raw_preview) > 0, f"Expected raw_results_preview non-empty. websearch={ws_dbg}"

first = raw_preview[0] or {}
assert isinstance(first, dict), f"Expected raw_results_preview items to be dicts. got={type(first)}"
assert first.get("url"), f"Expected URL in raw_results_preview[0]. raw_results_preview={raw_preview[:2]}"

print("WEBSEARCH OK")
print("Answer length:", len(answer_web.answer))
print("used_websearch:", route.used_websearch)
print("Websearch blocks:", ctx_blocks)
print("Websearch preview chars:", ws_dbg.get("context_preview_chars"))
print("Websearch no_evidence:", ws_dbg.get("no_evidence"))
print("Websearch docs preview:", ws_dbg.get("docs_preview"))
print("Websearch raw preview top urls:", [x.get("url") for x in raw_preview[:4]])


WEBSEARCH OK
Answer length: 818
Websearch blocks: 1
Websearch preview chars: 2537
Websearch docs preview: [{'title': 'Strategie projektowania promptów \xa0|\xa0 Gemini API \xa0|\xa0 Google AI for Developers', 'url': 'https://ai.google.dev/gemini-api/docs/prompting-strategies?hl=pl'}, {'title': 'JavaScript Jobs for December 2025 | Freelancer', 'url': 'https://www.freelancer.pl/jobs/javascript'}, {'title': 'HTML5 Jobs for December 2025 | Freelancer', 'url': 'https://www.freelancer.pl/jobs/html5'}, {'title': 'Relation 2025 - Data&AI Warsaw Tech Summit | Data&AI Warsaw Tech Summit', 'url': 'https://dataiwarsaw.tech/relation-2025/'}, {'title': 'Promocja urodzinowa Ebookpoint 2025 – Informatyka | Świat Czytników', 'url': 'https://swiatczytnikow.pl/ebookpoint-informatyka/'}]
